In [2]:
import requests
import time,math,json
import tagui as t
import re
from multiprocessing import Pool

<h4> Complete Singapore Address - From One Map </h4>

In [3]:
#adapt from https://github.com/xkjyeah/singapore-postal-codes
def pcode_to_data(pcode):
    if int(pcode) % 1000 == 0:
        print(pcode)
    
    page = 1
    results = []
    
    while True:
        try:
            response = requests.get('http://developers.onemap.sg/commonapi/search?searchVal={0}&returnGeom=Y&getAddrDetails=Y&pageNum={1}'
                                    .format(pcode, page)) \
                               .json()
        except requests.exceptions.ConnectionError as e:
            print('Fetching {} failed. Retrying in 2 sec'.format(pcode))
            
            time.sleep(2)
            continue
            
        results = results + response['results']
    
        if response['totalNumPages'] > page:
            page = page + 1
        else:
            break
            
    return results

In [ ]:
#if sufficient Memory 
if __name__ == '__main__':
    pool = Pool(processes=5)
    
    postal_codes = range(0, 1000000)
    postal_codes = ['{0:06d}'.format(p) for p in postal_codes]
    #postal_codes = ["119619","119620","499933","610410","225810","903808","307643","828273"]

    all_buildings = pool.map(pcode_to_data, postal_codes)
    all_buildings.sort(key=lambda b: (b['POSTAL'], b['SEARCHVAL']))

    jstr = json.dumps([y for x in all_buildings for y in x], indent=2, sort_keys=True)

    with open('address_list.json', 'w') as f:
        f.write(jstr.encode('utf-8'))

In [7]:
#if no sufficient memory
if __name__ == '__main__':
    #postal_codes = range(0, 1000000)
    #postal_codes = ['{0:06d}'.format(p) for p in postal_codes]
    postal_codes = ["119619","119620","499933","610410","225810","903808","307643","828273"," 574391","310066"]

    first = True

    with open('address_list.json', 'w') as f:
        for p in postal_codes:
          buildings = pcode_to_data(p)
          buildings.sort(key=lambda x: x['SEARCHVAL'])

          for building in buildings:
            if first:
              f.write('[\n')
              first = False
            else:
              f.write(',\n')
            print (building)
            f.write(re.sub('^', '  ', json.dumps(building, indent=2, sort_keys=True), flags=re.MULTILINE))

        f.write(']')

{'SEARCHVAL': '29 HENG MUI KENG TERRACE SINGAPORE 119620', 'BLK_NO': '29', 'ROAD_NAME': 'HENG MUI KENG TERRACE', 'BUILDING': 'NIL', 'ADDRESS': '29 HENG MUI KENG TERRACE SINGAPORE 119620', 'POSTAL': '119620', 'X': '21782.91399', 'Y': '30463.31192', 'LATITUDE': '1.291773265', 'LONGITUDE': '103.7774555', 'LONGTITUDE': '103.7774555'}
{'SEARCHVAL': 'ENERGY STUDIES INSTITUTE', 'BLK_NO': '29', 'ROAD_NAME': 'HENG MUI KENG TERRACE', 'BUILDING': 'ENERGY STUDIES INSTITUTE', 'ADDRESS': '29 HENG MUI KENG TERRACE ENERGY STUDIES INSTITUTE SINGAPORE 119620', 'POSTAL': '119620', 'X': '21787.01932', 'Y': '30331.1135', 'LATITUDE': '1.29057771', 'LONGITUDE': '103.77749240000001', 'LONGTITUDE': '103.77749240000001'}
{'SEARCHVAL': 'WORLD INTELLECTUAL PROPERTY ORGANISATION', 'BLK_NO': '29', 'ROAD_NAME': 'HENG MUI KENG TERRACE', 'BUILDING': 'WORLD INTELLECTUAL PROPERTY ORGANISATION', 'ADDRESS': '29 HENG MUI KENG TERRACE WORLD INTELLECTUAL PROPERTY ORGANISATION SINGAPORE 119620', 'POSTAL': '119620', 'X': '2184

<h4> Script for Zip Code </h4>

In [ ]:
#script to import all zip codes 
#t.close()
t.init(visual_automation = True)

with open('zipcode_list.csv', 'w', newline='',encoding="utf-8") as file: #create csv file 
    writer = csv.writer(file,delimiter=',')
    writer.writerow([ "Sno","Postal Code","Region"]) #create header
    sno = 0
    for i in range(0,1300):
        if i==0:
            t.url('https://sgp.postcodebase.com/all')
            t.wait(5)
        else:
            t.url(f'https://sgp.postcodebase.com/all?page={i}')
            t.wait(5)
        
        if t.present('//tbody//td'):
            total_items = int(t.count('//tbody//tr'))
            for j in range(1,total_items+1):
                d_temp =[]
                code = t.read(f'//tbody//tr[{j}]/td[1]')
                area = t.read(f'//tbody//tr[{j}]/td[2]')
                sno += 1
                d_temp = [sno,code, area]
                writer.writerow(d_temp) #write to csv file
                print (d_temp)
        else:
            break
t.close()